<a href="https://colab.research.google.com/github/namiyousef/argument-mining/blob/develop/experiments/yousef/OpenModelExperimentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Open Model Experimentation

This notebook allows you to play around with the models impemented :)

In [4]:
# -- CONFIGURATIONS


# to capture output - if install fails then remove to debug
%%capture 
!pip install argminer@git+https://github.com/namiyousef/argument-mining.git@main

# Set Model Configurations

In [25]:
from argminer.config import LABELS_MAP_DICT
from transformers import AutoTokenizer, AutoModelForTokenClassification
import pandas as pd

configuration = input('Select desired configuration. This should be of form: "{dataset} {model_name} {labelling strategy}')
dataset, model_name, strategy = configuration.split()
assert dataset in LABELS_MAP_DICT, "Please select dataset from 'Persuade, 'TUDarmstadt'"
assert model_name in ['bigBird', 'roberta'], "Please select model_name from 'bigBird', 'roberta'"
assert strategy in ['bio', 'io', 'bieo'], "Please select labelling strategy from 'bio', 'io', 'bieo'"

# inferred vars
short_dataset_name = {'Persuade': 'PER', 'TUDarmstadt': 'AAE'}[dataset]
hf_model_name = {'bigBird': 'google/bigbird-roberta-base', 'roberta':'roberta-base'}[model_name]
full_model_name = f'ucabqfe/{model_name}_{short_dataset_name}_{strategy}'

model = AutoModelForTokenClassification.from_pretrained(full_model_name)
tokenizer = AutoTokenizer.from_pretrained(hf_model_name, add_prefix_space=True)

df_label_map = LABELS_MAP_DICT[dataset][strategy]






Select desired configuration. This should be of form: "{dataset} {model_name} {labelling strategy}TUDarmstadt roberta bio


In [40]:
text = input('Please enter text segments and associated labels in the following format: {label}:{text_segment} -- {next_label},{next_text_segment}')
input_arr = text.split('--') # same as swagger but single dim
input_records = [item.split(':') for item in input_arr]

df_text = pd.DataFrame().from_records(input_records, columns=['text', 'label']).reset_index().rename(
    columns={'index':'doc_id'}
)


Please enter text segments and associated labels in the following format: {label}:{text_segment} -- {next_label},{next_text_segment}djsj: djajhfsbvj -- dsadj: jfsajfhds


In [44]:
max_length = 5
from argminer.data import ArgumentMiningDataset, TUDarmstadtProcessor, PersuadeProcessor

Processor = TUDarmstadtProcessor if dataset == 'TUDarmstadt' else PersuadeProcessor

import os

df_text.to_json(f'{dataset}_preprocessed.json')

print(Processor)
!ls
processor = Processor().from_json(status='preprocessed').process(strategy).postprocess()


df_text = processor.dataframe[['text', 'labels']]

dataset = ArgumentMiningDataset(df_label_map, df_text, tokenizer, max_length, f'standard_{strategy}', is_train=False)

<class 'argminer.data.TUDarmstadtProcessor'>
bio
df_TUDarmstadtProcessor_preprocessed
df_TUDarmstadtProcessor_preprocessed.json
sample_data
TUDarmstadt_preprocessed.json
TUDarmstadtProcessor_preprocessed
TUDarmstadtProcessor_preprocessed.json


KeyError: ignored

In [48]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
#model = AutoModelForTokenClassification.from_pretrained('ucabqfe/bigBird_PER_io')
tokenizer = AutoTokenizer.from_pretrained('ucabqfe/bigBird_PER_io')

OSError: ignored